In [ ]:
#mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install cftime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import xarray as xr

In [ ]:
centroid_data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/TA/SIDANG/DATA/centroid_area_under1k.xlsx')
centroid_data

,Centroid,geometry,tanggal,hari,bulan,tahun,date
0,1,POINT (100.49981049572239 -0.8650666211426935),24-7-2012,24,7,2012,2012-7-24
1,2,POINT (100.49413168718442 -0.8680840800275689),14-5-2015,14,5,2015,2015-5-14
2,3,POINT (100.27071241941778 -0.4574512281653418),24-7-2012,24,7,2012,2012-7-24
3,4,POINT (100.27071241941778 -0.4574512281653418),10-12-2018,10,12,2018,2018-12-10
4,5,POINT (100.18424377046533 -0.3120751452095314),9-1-2021,9,1,2021,2021-1-9
...,...,...,...,...,...,...,...
90,91,POINT (98.08347604465315 3.614368866728707),18-11-2020,18,11,2020,2020-11-18
91,92,POINT (98.03031174590635 3.6737872935545814),29-5-2016,29,5,2016,2016-5-29
92,93,POINT (100.46392124290186 0.0158685865110505),9-11-2020,9,11,2020,2020-11-9
93,94,POINT (100.61110773467394 0.0430482643908401),28-1-2011,28,1,2011,2011-1-28


In [ ]:
# Pisahkan Longitude dan Latitude
split = centroid_data["geometry"].str.split(" ", n = 2, expand=True)

# Rename kolom 
data_centroid = split.rename(columns = {1: "Longitude", 2: "Latitude"})

# Buang tanda ()
data_centroid['Longitude'] = data_centroid['Longitude'].str.replace("(", "").astype(float)
data_centroid['Latitude'] = data_centroid['Latitude'].str.replace(")","").astype(float)

data_centroid['tanggal'] = pd.to_datetime(centroid_data['tanggal'])

<ipython-input-5-8850de2c9259>:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data_centroid['Longitude'] = data_centroid['Longitude'].str.replace("(", "").astype(float)
<ipython-input-5-8850de2c9259>:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data_centroid['Latitude'] = data_centroid['Latitude'].str.replace(")","").astype(float)
<ipython-input-5-8850de2c9259>:11: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  data_centroid['tanggal'] = pd.to_datetime(centroid_data['tanggal'])


In [ ]:
data_centroid

,0,Longitude,Latitude,tanggal
0,POINT,100.499810,-0.865067,2012-07-24
1,POINT,100.494132,-0.868084,2015-05-14
2,POINT,100.270712,-0.457451,2012-07-24
3,POINT,100.270712,-0.457451,2018-10-12
4,POINT,100.184244,-0.312075,2021-09-01
...,...,...,...,...
90,POINT,98.083476,3.614369,2020-11-18
91,POINT,98.030312,3.673787,2016-05-29
92,POINT,100.463921,0.015869,2020-09-11
93,POINT,100.611108,0.043048,2011-01-28


In [ ]:
data_centroid.to_csv('/content/drive/MyDrive/Colab Notebooks/TA/SIDANG/DATA/Centroid95.csv', index=None)

In [ ]:
# Mengambil koordinat stasiun yang ada di Pulau Sumatera
lat = np.array(data_centroid['Latitude']).astype(np.float)
lon = np.array(data_centroid['Longitude']).astype(np.float)

min_lat = lat-0.05
min_lon = lon-0.05
max_lat = lat+0.05
max_lon = lon+0.05

<ipython-input-9-1d37181ccdb1>:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  lat = np.array(data_centroid['Latitude']).astype(np.float)
<ipython-input-9-1d37181ccdb1>:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  lon = np.array(data_centroid['Longitude']).astype(np.float)


In [ ]:
tabeltahun = []

for tahun in range(2000,2003):
  ds = xr.open_dataarray('/content/drive/MyDrive/Colab Notebooks/TA/Data GPM/GPM Indo {}.nc'.format(tahun), decode_times=False)
  
  ikoordinat = []
  for i in range(len(data_centroid)):
    ikoordinat.append(
        ds.sel(longitude=data_centroid.loc[i,'Longitude'],
              latitude=data_centroid.loc[i,'Latitude'],
              method='nearest'
              ).to_pandas().rename('centroid_{}'.format(i))
    )
  ds.close()
  tabeltahun.append(pd.concat(ikoordinat, axis=1))

df_alltime_allstat = pd.concat(tabeltahun, axis=0)
df_alltime_allstat

ValueError: ignored

In [ ]:
ds = xr.open_dataset('/content/drive/MyDrive/Colab Notebooks/TA/Data GPM/GPM Indo 2000.nc', decode_times=False)

In [ ]:
ds

<xarray.DataArray (time: 10272, longitude: 600, latitude: 301)>
[1855123200 values with dtype=float32]
Coordinates:
  * time       (time) int64 0 30 60 90 120 ... 308040 308070 308100 308130
  * longitude  (longitude) float32 90.05 90.15 90.25 90.35 ... 149.8 149.9 149.9
  * latitude   (latitude) float32 -15.05 -14.95 -14.85 ... 14.75 14.85 14.95

In [ ]:
ds = xr.open_dataset('/content/drive/MyDrive/Colab Notebooks/TA/Data GPM/GPM Indo 2000.nc', decode_times=False)

In [ ]:
df_alltime_allstat

,centroid_0,centroid_1,centroid_2,centroid_3,centroid_4,centroid_5,centroid_6,centroid_7,centroid_8,centroid_9,...,centroid_120,centroid_121,centroid_122,centroid_123,centroid_124,centroid_125,centroid_126,centroid_127,centroid_128,centroid_129
time,,,,,,,,,,,,,,,,,,,,,
2000-03-01 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
2000-03-01 01:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
2000-03-01 02:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
2000-03-01 03:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
2000-03-01 04:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31 19:00:00,4.649231,4.649231,15.974448,15.974448,15.583342,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.904517,0.000000,3.840051,0.000000
2021-12-31 20:00:00,7.580704,7.580704,15.421408,15.421408,16.129452,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.032383,3.514677,2.265864,1.656565
2021-12-31 21:00:00,2.881813,2.881813,1.197944,1.197944,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.184089,0.361910,0.000000,0.000000


In [ ]:
# Menyimpan data annual maxima
df_alltime_allstat.to_csv('/content/drive/MyDrive/Colab Notebooks/TA/Data GSMap/ts_centroid.csv', index=None)